# Nanosafety data query

In [15]:
# Setup
import ipywidgets as widgets
import json
import os,datetime

import logging
from logging.config import fileConfig
fileConfig('logging_endpoints_config.ini')
global logger
logger = logging.getLogger()
logger.debug('Started at %s \t%s',os.name, datetime.datetime.now())

try:
    with open('./config.solr.json') as f:
        solr_services = json.load(f)
except Exception as err:        
    solr_services = None
    logger.error("Something went wrong",err)

service_widget = widgets.Dropdown(
    options=solr_services.keys(),
    description='Service:',
    disabled=False,
)
display(service_widget)

apikey_widget=widgets.Text(
    placeholder='',
    description='API key',
    disabled=False
)
display(apikey_widget)


Dropdown(description='Service:', options=('NANoREG', 'eNanoMapper', 'NanoReg2', 'caLIBRAte', 'GRACIOUS', 'PATR…

Text(value='', description='API key', placeholder='')

In [16]:
term_widget=widgets.Text(
    placeholder='',
    description='term',
    disabled=False,
    value="E.method_s"
)
display(term_widget)

Text(value='E.method_s', description='term', placeholder='')

In [23]:
project=service_widget.value
solr_url=solr_services[project]
json_facet="{endpoint: {"+ "{}:{},{}:{} , limit:-1 ".format("type","terms","field",term_widget.value) + "}}"

#query = {"facet.field" : term_widget.value.strip() , "facet" : "on","rows" : 0 ,"q" : "*:*","wt" : "json"}
query={'q':"type_s:params","wt" : "json", "json.facet": json_facet, 'rows': 0}
logger.debug(solr_url)
logger.info(query)

2019-03-23 16:44:11,458  INFO     {'q': 'type_s:params', 'wt': 'json', 'json.facet': '{endpoint: {type:terms,field:E.method_s , limit:-1 }}', 'rows': 0}


In [24]:
import requests
import enmapi   
  
r = requests.post(solr_url,data=query, auth=enmapi.AmbitAPIKEYAuth(apikey_widget.value) ,stream=False)
logger.info(r.status_code)


2019-03-23 16:44:12,070  INFO     200


In [25]:
import pandas as pd
key = "endpoint"

fields = r.json()["facets"][key]["buckets"]
fields_name=[]
fields_count=[]
for value in fields:
    fields_name.append(value['val'])
    fields_count.append(value['count'])
        

df = pd.DataFrame({key : fields_name, "count" : fields_count})
(df)